In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My\ Drive/HW4

Mounted at /content/drive
/content/drive/My Drive/HW4


In [ ]:
# Setup for GLoVE embeddings (only need to run once)
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

# GLoVE Stuff

In [ ]:
import numpy as np
embeddings_dict={}
with open("glove.6B.50d.txt", 'r') as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_dict[word] = vector

In [ ]:
def get_embedding_features(raw_data, is_for_generation=False):
  """
  raw_data: List of tuples (words, label) corresponding to tweets and their corresponding hashtags
  is_for_generation: If using features for generation,
  return the sequence of word vectors, otherwise average over that dimension
  """
  featurized_data = []
  if not is_for_generation:
    vec = np.zeros((1,50))
  else:
    vec = []
  for row in raw_data:
    count = 0
    for word in row[0]:
      if not is_for_generation:
        if word in embeddings_dict:
          vec += embeddings_dict[word]
          count = count+1
      else:
        if word in embeddings_dict:
          vec.append(embeddings_dict[word])
    
    if not is_for_generation and count > 0:
      featurized_data.append([vec / count, row[1]])
    elif count > 0:
      featurized_data.append(np.stack(vec, axis=0))
  return featurized_data

# Hashtag Classification

In [ ]:
import os
import json
import re

hashtag_to_label = {
    'superbowl': 0,
    'sb49': 1,
    'patriots': 2,
    'nfl': 3,
    'gopatriots': 4,
    'gohawks': 5
}

datadir = "data"
raw_data = []
for fname in os.listdir(datadir):
  if not fname.endswith('.txt'):
    continue
  
  pound_idx = fname.index('#')
  hashtag = fname[pound_idx+1:-4]
  label_for_file = hashtag_to_label[hashtag]

  with open(os.path.join(datadir, fname)) as json_file:
    for line in json_file.readlines():
      data_pt = json.loads(line)
      
      tweet = data_pt['tweet']['text']
      words = tweet.split()
      words = [x for x in words if x[0] != '#' and not x.startswith('http')] # remove hashtags and urls from tweet
      words = [re.sub(r'[^\w\s]', '', x) for x in words] # Remove punctuation
      
      raw_data.append([words, label_for_file])

In [ ]:
classification_feats = get_embedding_features(raw_data, is_for_generation=False)
classification_labels = np.array([x[1] for x in classification_feats])
classification_feats = np.concatenate([x[0] for x in classification_feats], axis=0)
print(classification_feats.shape, classification_labels.shape)
# import numpy as np
# classification_feats = np.load("features.npy")
# classification_labels = np.load("labels.npy")

(2495446, 50) (2495446,)


In [ ]:
np.save('features.npy', classification_feats)
np.save('labels.npy', classification_labels)

In [ ]:
import torch.nn as nn

class MLP(nn.Module):
  def __init__(self, num_hidden_layers, hidden_width, input_width, output_width):
    super().__init__()
    self.input_layer = nn.Linear(input_width, hidden_width)
    self.relu = nn.ReLU()

    hidden_layer_list = []
    for i in range(num_hidden_layers):
      hidden_layer_list.append(nn.Linear(hidden_width, hidden_width))
      hidden_layer_list.append(nn.ReLU())
    self.hidden_layers = nn.Sequential(*hidden_layer_list)

    self.output_layer = nn.Linear(hidden_width, output_width)
  
  def forward(self, x):
    x = self.input_layer(x)
    x = self.relu(x)
    x = self.hidden_layers(x)
    return self.output_layer(x)

In [ ]:
from sklearn.model_selection import train_test_split
import torch

model = MLP(num_hidden_layers=6, hidden_width=128, input_width=50, output_width=6) # 6 output classes corresponding to each hashtag
x_train, x_test, y_train, y_test = train_test_split(classification_feats, classification_labels, test_size=0.1)

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128
num_epochs = 1

model.cuda() # moves the model to the GPU

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

torch.Size([2245901, 50]) torch.Size([249545, 50]) torch.Size([2245901]) torch.Size([249545])


In [ ]:
from tqdm import tqdm
# Helper function to print validation loss/accuracy
def evaluate(model, x_test, y_test, epoch):
  model.eval()
  with torch.no_grad():
    num_correct = 0.0
    for i in range(0, x_test.shape[0], batch_size):
      if i+batch_size > x_test.shape[0]:
        i_end = x_test.shape[0]
      else:
        i_end = i + batch_size

      model_in = x_test[i:i_end].cuda()
      labels = y_test[i:i_end].cuda()

      preds = model(model_in)
      num_correct += torch.sum(torch.argmax(preds, dim=1) == labels)
  print(f"Epoch: {epoch}, Validation Accuracy: {num_correct / x_test.shape[0]}")
  model.train()

evaluate(model, x_test, y_test, -1)
# Main training loop
for epoch in range(num_epochs):
  avg_loss = 0.0
  count = 0
  for i in tqdm(range(0, x_train.shape[0], batch_size)):
    if i+batch_size > x_train.shape[0]:
      i_end = x_train.shape[0]
    else:
      i_end = i + batch_size
  
    model_in = x_train[i:i_end].cuda() # Gets one batch of training data and GT labels onto the GPU
    labels = y_train[i:i_end].type('torch.cuda.LongTensor')

    # Forward pass through the model
    model_out = model(model_in)

    # Backwards pass to calculate gradients and update parameters
    loss = loss_fn(model_out, labels)
    avg_loss += loss.item()
    count += 1
    # print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f"Average Training Loss for Epoch {epoch}: {avg_loss / count}")
  evaluate(model, x_test, y_test, epoch)


Epoch: -1, Validation Accuracy: 0.15814782679080963


100%|██████████| 17547/17547 [01:07<00:00, 261.13it/s]


Average Training Loss for Epoch 0: 5.479392735832636
Epoch: 0, Validation Accuracy: 0.4214750826358795


# Tweet Generation

In [ ]:
import os
import json
import re

datadir = "data"
fname = "tweets_#gopatriots.txt" # Change this to load data for a different hashtag
raw_data = []
full_tweets = []
with open(os.path.join(datadir, fname)) as json_file:
  for line in json_file.readlines():
    data_pt = json.loads(line)
      
    tweet = data_pt['tweet']['text']
    words = tweet.split()
    words = [x.lower() for x in words if x[0] != '#' and not x.startswith('http')] # remove hashtags and urls from tweet
    words = [re.sub(r'[^\w\s]', '', x) for x in words] # Remove punctuation
    words = [x for x in words if len(x) > 0]
    if len(words) < 2:
      continue

    full_tweets.append(words)
      
    for i in range(len(words)-1):
      raw_data.append((words[:i+1], words[i+1]))

In [ ]:
print(len(raw_data))
print(len(full_tweets))

128957
18731


In [ ]:
!pip install pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 9.5 MB/s eta 0:00:00


In [ ]:
from torchnlp.encoders.text import StaticTokenizerEncoder

# Creates the vocabulary over all the tweets
encoder = StaticTokenizerEncoder(full_tweets, tokenize=lambda x:x, min_occurrences=10)
print(len(encoder.vocab))

# Tokenizes each tweet in the dataset
processed_data = []
for tweet in full_tweets:
  tokenized = encoder.encode(tweet)

  input = tokenized[:-1]
  label = tokenized[-1]

  processed_data.append((input, label))

# Caluclates max sequence length for use in padding during training
sequence_length = max([x[0].shape[0] for x in processed_data])
print(sequence_length)

1554
27


In [ ]:
import torch.nn as nn
import torch
class Generator(nn.Module):
  def __init__(self, vocab_size, hidden_dim=128, num_lstm_layers=3):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
    self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_lstm_layers, batch_first=True)
    self.output_layer = nn.Linear(hidden_dim, vocab_size)

  
  def forward(self, x, in_state=None):
    x = self.embedding(x)
    x, state = self.lstm(x)
    return self.output_layer(x), state

def evaluate(model, epoch, test_data, batch_size):
  model.eval()
  num_correct = 0
  for i in tqdm(range(0, len(test_data), batch_size)):
    if i+batch_size > len(test_data):
      i_end = len(test_data)
    else:
      i_end = i+batch_size

    batch = test[i:i_end]

    padded = [pad_tensor(x[0], length=sequence_length) for x in batch] # Necessary to pad each sequence so they are all the same length
    model_in = torch.stack(padded, dim=0).cuda()
    labels = torch.stack([x[1] for x in batch], dim=0).type('torch.cuda.LongTensor')

    with torch.no_grad():
      preds, _ = model(model_in)

    num_correct += torch.sum(torch.argmax(preds[:,-1,:], dim=-1) == labels)
  print(f"Validation Accuracy for epoch {epoch}: {num_correct / len(test_data)}")
  model.train()

In [ ]:
from torchnlp.encoders.text import pad_tensor
from tqdm import tqdm
from sklearn.model_selection import train_test_split

model = Generator(len(encoder.vocab)).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50
batch_size = 64

train, test = train_test_split(processed_data, test_size=0.2)

state = (torch.zeros((3, batch_size, 128)).cuda(), torch.zeros((3, batch_size, 128)).cuda())
for epoch in range(num_epochs):
  average_loss = 0.0
  for i in tqdm(range(0, len(train), batch_size)):
    if i+batch_size > len(train):
      continue
    else:
      i_end = i+batch_size
    
    batch = train[i:i_end]

    padded = [pad_tensor(x[0], length=sequence_length) for x in batch] # Necessary to pad each sequence so they are all the same length
    model_in = torch.stack(padded, dim=0).cuda()
    labels = torch.stack([x[1] for x in batch], dim=0).type('torch.cuda.LongTensor')

    preds, state = model(model_in)

    # state = (state[0].detach(), state[1].detach())

    loss = loss_fn(preds[:,-1,:], labels)
    average_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f"Average Loss for epoch {epoch}: {average_loss / len(train)}")
  evaluate(model, epoch, test, batch_size)

100%|██████████| 235/235 [00:02<00:00, 101.12it/s]


Average Loss for epoch 0: 0.07795519831660465


100%|██████████| 59/59 [00:00<00:00, 546.35it/s]


Validation Accuracy for epoch 0: 0.29917266964912415


100%|██████████| 235/235 [00:01<00:00, 163.23it/s]


...

Validation Accuracy for epoch 48: 0.31812116503715515


100%|██████████| 235/235 [00:01<00:00, 163.65it/s]


Average Loss for epoch 49: 0.04925700512978716


100%|██████████| 59/59 [00:00<00:00, 467.39it/s]

Validation Accuracy for epoch 49: 0.3101147711277008


**GRU**

In [ ]:
class Generator_GRU1(nn.Module):
  def __init__(self, vocab_size, hidden_dim=128, num_lstm_layers=3):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
    self.lstm = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_lstm_layers, batch_first=True)
    self.output_layer = nn.Linear(hidden_dim, vocab_size)

  
  def forward(self, x, in_state=None):
    x = self.embedding(x)
    x, state = self.lstm(x)
    return self.output_layer(x), state

class Generator_GRU2(nn.Module):
  def __init__(self, vocab_size, hidden_dim=128, num_gru_layers=3):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
    self.gru1 = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_gru_layers, batch_first=True)
    self.gru2 = nn.GRU(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=num_gru_layers, batch_first=True)
    self.output_layer1 = nn.Linear(hidden_dim, hidden_dim)
    self.output_layer2 = nn.Linear(hidden_dim, vocab_size)

  
  def forward(self, x, in_state=None):
    x = self.embedding(x)
    x, state1 = self.gru1(x)
    x, state2 = self.gru2(x)
    x = self.output_layer1(x)
    return self.output_layer2(x), state2

In [ ]:
model = Generator_GRU1(len(encoder.vocab)).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50
batch_size = 64

train, test = train_test_split(processed_data, test_size=0.2)

state = (torch.zeros((3, batch_size, 128)).cuda(), torch.zeros((3, batch_size, 128)).cuda())
for epoch in range(num_epochs):
  average_loss = 0.0
  for i in tqdm(range(0, len(train), batch_size)):
    if i+batch_size > len(train):
      continue
    else:
      i_end = i+batch_size
    
    batch = train[i:i_end]

    padded = [pad_tensor(x[0], length=sequence_length) for x in batch] # Necessary to pad each sequence so they are all the same length
    model_in = torch.stack(padded, dim=0).cuda()
    labels = torch.stack([x[1] for x in batch], dim=0).type('torch.cuda.LongTensor')

    preds, state = model(model_in)

    # state = (state[0].detach(), state[1].detach())

    loss = loss_fn(preds[:,-1,:], labels)
    average_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f"Average Loss for epoch {epoch}: {average_loss / len(train)}")
  evaluate(model, epoch, test, batch_size)

100%|██████████| 235/235 [00:01<00:00, 167.84it/s]


Average Loss for epoch 0: 0.07833673234171748


100%|██████████| 59/59 [00:00<00:00, 591.91it/s]


Validation Accuracy for epoch 0: 0.31838804483413696


100%|██████████| 235/235 [00:01<00:00, 163.63it/s]


...

Validation Accuracy for epoch 48: 0.27782225608825684


100%|██████████| 235/235 [00:01<00:00, 185.62it/s]


Average Loss for epoch 49: 0.006064862510098917


100%|██████████| 59/59 [00:00<00:00, 541.91it/s]

Validation Accuracy for epoch 49: 0.28049105405807495


In [ ]:
model = Generator_GRU2(len(encoder.vocab)).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 50
batch_size = 64

train, test = train_test_split(processed_data, test_size=0.2)

state = (torch.zeros((3, batch_size, 128)).cuda(), torch.zeros((3, batch_size, 128)).cuda())
for epoch in range(num_epochs):
  average_loss = 0.0
  for i in tqdm(range(0, len(train), batch_size)):
    if i+batch_size > len(train):
      continue
    else:
      i_end = i+batch_size
    
    batch = train[i:i_end]

    padded = [pad_tensor(x[0], length=sequence_length) for x in batch] # Necessary to pad each sequence so they are all the same length
    model_in = torch.stack(padded, dim=0).cuda()
    labels = torch.stack([x[1] for x in batch], dim=0).type('torch.cuda.LongTensor')

    preds, state = model(model_in)

    # state = (state[0].detach(), state[1].detach())

    loss = loss_fn(preds[:,-1,:], labels)
    average_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f"Average Loss for epoch {epoch}: {average_loss / len(train)}")
  evaluate(model, epoch, test, batch_size)

100%|██████████| 235/235 [00:02<00:00, 101.60it/s]


Average Loss for epoch 0: 0.0780502396093227


100%|██████████| 59/59 [00:00<00:00, 225.32it/s]


Validation Accuracy for epoch 0: 0.3146517276763916


100%|██████████| 235/235 [00:01<00:00, 120.30it/s]


...

Validation Accuracy for epoch 48: 0.22951695322990417


100%|██████████| 235/235 [00:01<00:00, 119.24it/s]


Average Loss for epoch 49: 0.009942473262922368


100%|██████████| 59/59 [00:00<00:00, 434.47it/s]

Validation Accuracy for epoch 49: 0.24686415493488312


In [ ]:
class MarkovModel:
   # Model from https://github.com/thomashikaru
   
    """Represents a Markov Model for a given text"""

    def __init__(self, n, text):
        """Constructor takes n-gram length and training text
        and builds dictionary mapping n-grams to
        character-probability mappings."""
        self.n = n
        self.d = {}
        for i in range(len(text)-n-1):
            ngram = text[i:i+n]
            nextchar = text[i+n:i+n+1]
            if ngram in self.d:
                if nextchar in self.d[ngram]:
                    self.d[ngram][nextchar] += 1
                else:
                    self.d[ngram][nextchar] = 1
            else:
                self.d[ngram] = {nextchar: 1}

    def test_init(self):
        for x in (list(self.d.items())[:10]):
            print(x)

    def get_next_char(self, ngram):
        """Generates a single next character based to come after the provided n-gram,
        based on the probability distribution learned from the text."""
        if ngram in self.d:
            dist = self.d[ngram]
            distlist = list(dist.items())
            keys = [k for k, _ in distlist]
            vals = [v for _, v in distlist]
            valsum = sum(vals)
            vals = list(map(lambda x: x/valsum, vals))
            return np.random.choice(keys, 1, p=vals)[0]
        else:
            # this should never happen if start string n-gram exists in train text
            return np.random.choice([x for x in "abcdefghijklmnopqrstuvwxyz"])

    def get_n_chars(self, length, ngram):
        """Returns a generated sequence of specified length,
        using the given n-gram as a starting seed."""
        s = []
        for i in range(length):
            nextchar = self.get_next_char(ngram)
            ngram = ngram[1:]+nextchar
            s.append(nextchar)
        return ''.join(s)

In [ ]:
#raw_data
#full_tweets

In [ ]:
ngram_length = 4
tweet_length = 280
model = MarkovModel(ngram_length, text)
initial_ngram = "Hill"[:ngram_length]
print(initial_ngram + model.get_n_chars(tweet_length, initial_ngram))

Hillary of @lisall be dignific people sure and fair unsel/Justing! The specialMelB &amp; over express security - realDonal law in term(s) in Marylanded from the five-Disgrace would site House present. on @thehill poll Trump....youtubes making give Voice....Senator the race. The Afric


In [ ]:
tweet_text = ''
for tweets in raw_data:
  for tweet in tweets:
    #print(type(tweet[0][0]))
    temp = " ".join(tweet)
    tweet_text = tweet_text + temp

In [ ]:
ngram_length = 4
tweet_length = 280
model = MarkovModel(ngram_length, tweet_text)
initial_ngram = "Football"[:ngram_length]
print(initial_ngram + model.get_n_chars(tweet_length, initial_ngram))

Footro see the supongod som a k ich one were we are bet when montra closersn oway timelisto para el y nobodyt obrey o u s e r s a e c klord day should be up a ti brady te d o wso walking louie now days scome goes thanks for jonás que eut einteres about to keeping fun brady gronkwoski
